**Import required libraries and scripts**

In [ ]:
import os
for dir in os.listdir('/media/mario/T7/FINISHED/FINISHED2/'):
    print(dir)

In [1]:
import os
from scripts.library_preparation import *
from scripts.utilities import *
from scripts.docking_functions import *
from scripts.clustering_functions import *
from scripts.rescoring_functions import *
from scripts.ranking_functions import *
from scripts.performance_calculation import *
from scripts.dogsitescorer import *
from scripts.get_pocket import *

for dir in os.listdir('/media/mario/T7/FINISHED/FINISHED2/'):
    print(dir)
    software = '/home/mario/DockM8/software'
    protein_file = f'/media/mario/T7/FINISHED/FINISHED2/{dir}/receptor_protoss_prepared.pdb'
    ref_file = f'/media/mario/T7/FINISHED/FINISHED2/{dir}/crystal_ligand_protoss.sdf'
    pocket = 'reference'
    protonation = 'pkasolver'
    docking_library = f'/media/mario/T7/FINISHED/FINISHED2/{dir}/merged_actives_decoys.sdf'
    docking_programs = ['GNINA', 'SMINA', 'PLANTS']
    clustering_metrics = ['RMSD', 'spyRMSD', 'espsim', '3DScore', 'bestpose', 'bestpose_GNINA', 'bestpose_SMINA', 'bestpose_PLANTS']
    clustering_method = 'KMedoids'
    rescoring= ['gnina', 'AD4', 'chemplp', 'rfscorevs', 'LinF9', 'SCORCH', 'RTMScore', 'vinardo']
    id_column = 'ID'
    n_poses = 10
    exhaustiveness = 8
    parallel = 1
    ncpus = int(os.cpu_count()/2)
    #Create a temporary folder for all further calculations
    w_dir = os.path.dirname(protein_file)
    print('The working directory has been set to:', w_dir)
    create_temp_folder(w_dir+'/temp')
    
    try:   
        for metric in clustering_metrics:
            rescore_all(w_dir, protein_file, ref_file, software, w_dir+f'/temp/clustering/{metric}_clustered.sdf', rescoring, parallel, ncpus)
            
        apply_consensus_methods_combinations(w_dir, clustering_metrics)       
    except:
        printlog(f'Failed for {dir}')


[09:03:26] Initializing Normalizer
/home/mario/anaconda3/envs/wocondock/lib/python3.8/site-packages/MDAnalysis/coordinates/chemfiles.py:108: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  MIN_CHEMFILES_VERSION = LooseVersion("0.9")
[TRJ.py:171 - <module>()] netCDF4 is not available. Writing AMBER ncdf files will be slow.


hs90a
The working directory has been set to: /media/mario/T7/FINISHED/FINISHED2/hs90a
The folder: /media/mario/T7/FINISHED/FINISHED2/hs90a/temp already exists
The folder: /media/mario/T7/FINISHED/FINISHED2/hs90a/temp/rescoring_RMSD_clustered already exists

[2023-Mar-29 09:03:29]: /gnina_rescoring folder already exists, skipping gnina rescoring

[2023-Mar-29 09:03:29]: /AD4_rescoring folder already exists, skipping AD4 rescoring

[2023-Mar-29 09:03:29]: /chemplp_rescoring folder already exists, skipping chemplp rescoring

[2023-Mar-29 09:03:29]: /rfscorevs_rescoring folder already exists, skipping rfscorevs rescoring

[2023-Mar-29 09:03:29]: /LinF9_rescoring folder already exists, skipping LinF9 rescoring

[2023-Mar-29 09:03:29]: /SCORCH_rescoring folder already exists, skipping SCORCH rescoring

[2023-Mar-29 09:03:29]: /RTMScore_rescoring folder already exists, skipping RTMScore rescoring


/home/mario/anaconda3/envs/wocondock/lib/python3.8/site-packages/MDAnalysis/coordinates/TRJ.py:1209: DeprecationWarning: Please use `netcdf_file` from the `scipy.io` namespace, the `scipy.io.netcdf` namespace is deprecated.
  class NCDFPicklable(scipy.io.netcdf.netcdf_file):


'/media/mario/T7/FINISHED/FINISHED2/hs90a/temp/clustering/RMSD_clustered.sdf'


[2023-Mar-29 09:03:29]: Rescoring with Vinardo

[2023-Mar-29 09:03:50]: Rescoring with Vinardo complete in 21.0882!

[2023-Mar-29 09:03:50]: Combining all score for /media/mario/T7/FINISHED/FINISHED2/hs90a/temp/rescoring_RMSD_clustered


...

'/media/mario/T7/FINISHED/FINISHED2/hxk4/temp/clustering/bestpose_SMINA_clustered.sdf'


[2023-Mar-29 11:37:17]: Rescoring with Vinardo

[2023-Mar-29 11:37:27]: Rescoring with Vinardo complete in 9.9601!

[2023-Mar-29 11:37:27]: Combining all score for /media/mario/T7/FINISHED/FINISHED2/hxk4/temp/rescoring_bestpose_SMINA_clustered


Combining scores: 100%|██████████| 7/7 [00:00<00:00, 238.86files/s]


[2023-Mar-29 11:37:28]: Rescoring complete in 1096.7263!
The folder: /media/mario/T7/FINISHED/FINISHED2/hxk4/temp/rescoring_bestpose_PLANTS_clustered already exists

[2023-Mar-29 11:37:28]: /gnina_rescoring folder already exists, skipping gnina rescoring

[2023-Mar-29 11:37:28]: /AD4_rescoring folder already exists, skipping AD4 rescoring

[2023-Mar-29 11:37:28]: /chemplp_rescoring folder already exists, skipping chemplp rescoring

[2023-Mar-29 11:37:28]: /rfscorevs_rescoring folder already exists, skipping rfscorevs rescoring

[2023-Mar-29 11:37:28]: /LinF9_rescoring folder already exists, skipping LinF9 rescoring
The folder: /media/mario/T7/FINISHED/FINISHED2/hxk4/temp/rescoring_bestpose_PLANTS_clustered/SCORCH_rescoring/ was created

[2023-Mar-29 11:37:28]: Converting protein file to .pdbqt ...



[2023-Mar-29 11:37:48]: Converting SDF file bestpose_PLANTS_clustered.sdf to .pdbqt files...
Converted 4338 molecules.

[2023-Mar-29 11:37:58]: Rescoring with SCORCH
